In [1]:
import pandas as pd
import numpy as np
import requests
df = pd.read_csv('广州写字楼数据.csv',error_bad_lines=False)

In [2]:
def replacestr(string):
    string = string.replace('[','')
    string = string.replace(']','')
    string = string.replace("'",'')
    return string

df['loupandizhi'] = df['loupandizhi'].apply(replacestr)

list_location = df['loupandizhi'].value_counts().index.tolist()

dict_location = {}
Key = '6b95b84cab6100c0a6f52c62ece97e4b'
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in list_location:
    location = i.split(",")[-1].strip()
    url = url_pre.format(Key, location, '广州')
    res = requests.get(url).json()
    if res['status'] == '0':
        dict_location[i] = np.nan
        counter +=1
    else:
        try: 
            dict_location[i] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            dict_location[i] = np.nan
            
        counter +=1

one batch(250) processed
one batch(250) processed


In [3]:
df['location_coords'] = np.nan

counter = 0
for i in list_location:
    counter += 1
    df_temp = df[df['loupandizhi'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,15] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [4]:
df.head()

,building_type,building_year,chuzumianji,floor,loupan_content,loupandizhi,price_perunit,release_date,service_price,titile,total_price_10K_permonth,url,xiezilou_level,xzl_content,zhuangxiu,location_coords
0,纯写字楼,NaN,373㎡,低区(共40层),\r\n \r\n \r\n ...,"琶洲, 保利中悦广场",145.04,2019/9/3 10:51:52,NaN,新盘上线！保利中悦广场 全新装修 东南向 带有3个间隔,54100,https://gz.office.fang.com/zu/3_241955883.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,NaN,NaN
1,纯写字楼,NaN,150㎡,中区(共37层),\r\n \r\n \r\n ...,"珠江新城西, 保利克洛维广场",150.00,2019/9/6 0:47:15,物业费20.00元/平米·月,珠江新城 保利克洛维150方 3个间隔豪装配家私,22500,https://gz.office.fang.com/zu/3_242041826.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,精装修(可分割),"113.320528,23.126984"
2,纯写字楼,NaN,272㎡,低区(共41层),\r\n \r\n \r\n ...,"珠江新城中, 富力盈通大厦",150.00,2019/9/5 18:25:13,物业费24.00元/平米·月,富力盈通大厦 272方 一手业主直租 精装大前台,40800,https://gz.office.fang.com/zu/3_242032603.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,精装修(不可分割),"113.321956,23.126051"
3,纯写字楼,NaN,150㎡,高区(共68层),\r\n \r\n \r\n ...,"珠江新城中, 环球都会广场",150.00,2019/8/8 9:59:29,物业费35.00元/平米·月,南向望小蛮腰 环球都会广场150方 高层对电梯 带豪带全套家私,22500,https://gz.office.fang.com/zu/3_241234419.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,精装修(不可分割),"113.327273,23.118458"
4,纯写字楼,NaN,2000㎡,中区(共30层),\r\n \r\n \r\n ...,"珠江新城西, 保利克洛维广场",150.00,2019/9/2 15:46:43,物业费16.00元/平米·月,保利克洛维大厦 双地铁全天空调 2000方仅租150元,300000,https://gz.office.fang.com/zu/3_241939800.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,NaN,"113.320528,23.126984"


In [5]:
df.to_csv('广州写字楼出租地址数据.csv', encoding = 'utf-8-sig',index = False)